<a href="https://colab.research.google.com/github/EmilyCarroll-del/Predicting-Freezing-of-Gait-in-Parkinson-s-Disease-Using-ML/blob/main/ResNetFinal_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn.model_selection import StratifiedGroupKFold

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd '/content/drive/MyDrive/Colab Notebooks/'
%ls

/content/drive/MyDrive/Colab Notebooks
 FFT.ipynb                                 ResNetNew.ipynb
 RandomForestFinalPreprocessedData.ipynb   Test-Data/
'Random Forest_Its_broken_now.ipynb'       tlvmc-parkinsons-freezing-gait-prediction/
 RandomForestNew.ipynb                     Training-Data/
 ResNetFinal.ipynb.ipynb                   Untitled0.ipynb
 ResNet.ipynb                              Untitled1.ipynb
 ResNetNewBasicModel.ipynb


In [4]:
def process_directory(directory):
    dfs = []
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)

        # Check if the file is a CSV and process it
        if file_name.endswith('.csv') and os.path.isfile(file_path):
            df = pd.read_csv(file_path) #read csv files
            df['source_directory'] = os.path.basename(directory)  # Add a column to identify the source directory
            df['csv_name'] = os.path.basename(file_name)
            dfs.append(df)

    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

# Load Data from CSV files

In [5]:
#directory to each folder in train
train_defog_dir = '/content/drive/MyDrive/Colab Notebooks/tlvmc-parkinsons-freezing-gait-prediction/train/defog'
train_tdcsfog_dir = '/content/drive/MyDrive/Colab Notebooks/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog'

#train_notype_dir = '/content/drive/MyDrive/Colab Notebooks/tlvmc-parkinsons-freezing-gait-prediction/train/notype'

In [6]:
# Process each subdirectory
defog = process_directory(train_defog_dir)
tdcsfog = process_directory(train_tdcsfog_dir)
#notype = process_directory(train_notype_dir)

#train_df = pd.concat(train_data, ignore_index=True)
print("Data concatenation complete.")

Data concatenation complete.


In [7]:
#Test CSV files
defog_test_dir = '/content/drive/MyDrive/Colab Notebooks/tlvmc-parkinsons-freezing-gait-prediction/test/defog.csv'
tdcsfog_test_dir = '/content/drive/MyDrive/Colab Notebooks/tlvmc-parkinsons-freezing-gait-prediction/test/tdcsfog.csv'

In [8]:
#csv files -> df
defog_test = pd.read_csv(defog_test_dir)
tdcsfog_test = pd.read_csv(tdcsfog_test_dir)

# Data preprocessing

In [9]:
#removing missing values of defog test and training data
#defog dataset, valid = false, task = false; tdcsfog has no missing values
defog = defog[defog['Valid']]
defog = defog[defog['Task']]

defog_test = defog_test[defog_test['Valid']]
defog_test = defog_test[defog_test['Task']]

In [10]:
print(defog.head())
print(defog_test.head())

      Time      AccV     AccML     AccAP  StartHesitation  Turn  Walking  \
1000  1000 -0.970018  0.061626 -0.265625                0     0        0   
1001  1001 -0.984375  0.044497 -0.265625                0     0        0   
1002  1002 -0.984375  0.029016 -0.265625                0     0        0   
1003  1003 -0.984375  0.015625 -0.265625                0     0        0   
1004  1004 -0.984670  0.015330 -0.265625                0     0        0   

      Valid  Task source_directory        csv_name  
1000   True  True            defog  02ea782681.csv  
1001   True  True            defog  02ea782681.csv  
1002   True  True            defog  02ea782681.csv  
1003   True  True            defog  02ea782681.csv  
1004   True  True            defog  02ea782681.csv  
      Time      AccV     AccML     AccAP  StartHesitation  Turn  Walking  \
1000  1000 -0.976776 -0.104293  0.182096                0     0        0   
1001  1001 -0.977049 -0.106393  0.182613                0     0        0 

In [11]:
#drop a small amount of outliers -defog train data

from scipy.stats import zscore

# Calculate z-scores for each accelerometer axis
defog["AccV_z"] = zscore(defog["AccV"])
defog["AccML_z"] = zscore(defog["AccML"])
defog["AccAP_z"] = zscore(defog["AccAP"])

# Set threshold range for z-scores (-5 < z < 5)
threshold = 5

# Find rows where any component exceeds the threshold
outliers = (defog["AccV_z"].abs() > threshold) | \
           (defog["AccML_z"].abs() > threshold) | \
           (defog["AccAP_z"].abs() > threshold)

# Filter out these rows from the dataset
defog = defog[~outliers].drop(columns=["AccV_z", "AccML_z", "AccAP_z"])

# Save or inspect the filtered DataFrame
print("Number of points dropped:", outliers.sum())
print(defog.head())

Number of points dropped: 11644
      Time      AccV     AccML     AccAP  StartHesitation  Turn  Walking  \
1000  1000 -0.970018  0.061626 -0.265625                0     0        0   
1001  1001 -0.984375  0.044497 -0.265625                0     0        0   
1002  1002 -0.984375  0.029016 -0.265625                0     0        0   
1003  1003 -0.984375  0.015625 -0.265625                0     0        0   
1004  1004 -0.984670  0.015330 -0.265625                0     0        0   

      Valid  Task source_directory        csv_name  
1000   True  True            defog  02ea782681.csv  
1001   True  True            defog  02ea782681.csv  
1002   True  True            defog  02ea782681.csv  
1003   True  True            defog  02ea782681.csv  
1004   True  True            defog  02ea782681.csv  


In [12]:
#drop a small amount of outliers -defog test

from scipy.stats import zscore

# Calculate z-scores for each accelerometer axis
defog_test["AccV_z"] = zscore(defog_test["AccV"])
defog_test["AccML_z"] = zscore(defog_test["AccML"])
defog_test["AccAP_z"] = zscore(defog_test["AccAP"])

# Set threshold range for z-scores (-5 < z < 5)
threshold = 5

# Find rows where any component exceeds the threshold
outliers = (defog_test["AccV_z"].abs() > threshold) | \
           (defog_test["AccML_z"].abs() > threshold) | \
           (defog_test["AccAP_z"].abs() > threshold)

# Filter out these rows from the dataset
defog_test = defog_test[~outliers].drop(columns=["AccV_z", "AccML_z", "AccAP_z"])

# Save or inspect the filtered DataFrame
print("Number of points dropped:", outliers.sum())
print(defog_test.head())

Number of points dropped: 349
      Time      AccV     AccML     AccAP  StartHesitation  Turn  Walking  \
1000  1000 -0.976776 -0.104293  0.182096                0     0        0   
1001  1001 -0.977049 -0.106393  0.182613                0     0        0   
1002  1002 -0.975982 -0.106688  0.182188                0     0        0   
1003  1003 -0.904200 -0.080405  0.173685                0     0        0   
1004  1004 -0.985695 -0.114734  0.192869                0     0        0   

      Valid  Task  
1000   True  True  
1001   True  True  
1002   True  True  
1003   True  True  
1004   True  True  


In [13]:
#drop a small amount of outliers -tdcsfog training

# Calculate z-scores for each accelerometer axis
tdcsfog["AccV_z"] = zscore(tdcsfog["AccV"])
tdcsfog["AccML_z"] = zscore(tdcsfog["AccML"])
tdcsfog["AccAP_z"] = zscore(tdcsfog["AccAP"])

# Set threshold range for z-scores (-5 < z < 5)
threshold = 5

# Find rows where any component exceeds the threshold
outliers = (tdcsfog["AccV_z"].abs() > threshold) | \
           (tdcsfog["AccML_z"].abs() > threshold) | \
           (tdcsfog["AccAP_z"].abs() > threshold)

# Filter out these rows from the dataset
tdcsfog = tdcsfog[~outliers].drop(columns=["AccV_z", "AccML_z", "AccAP_z"])

# Save or inspect the filtered DataFrame
print("Number of points dropped:", outliers.sum())
print(tdcsfog.head())

Number of points dropped: 27184
   Time      AccV     AccML     AccAP  StartHesitation  Turn  Walking  \
0     0 -9.533939  0.566322 -1.413525                0     0        0   
1     1 -9.536140  0.564137 -1.440621                0     0        0   
2     2 -9.529345  0.561765 -1.429332                0     0        0   
3     3 -9.531239  0.564227 -1.415490                0     0        0   
4     4 -9.540825  0.561854 -1.429471                0     0        0   

  source_directory        csv_name  
0          tdcsfog  003f117e14.csv  
1          tdcsfog  003f117e14.csv  
2          tdcsfog  003f117e14.csv  
3          tdcsfog  003f117e14.csv  
4          tdcsfog  003f117e14.csv  


In [14]:
#drop a small amount of outliers -tdcsfog test

# Calculate z-scores for each accelerometer axis
tdcsfog_test["AccV_z"] = zscore(tdcsfog_test["AccV"])
tdcsfog_test["AccML_z"] = zscore(tdcsfog_test["AccML"])
tdcsfog_test["AccAP_z"] = zscore(tdcsfog_test["AccAP"])

# Set threshold range for z-scores (-5 < z < 5)
threshold = 5

# Find rows where any component exceeds the threshold
outliers = (tdcsfog_test["AccV_z"].abs() > threshold) | \
           (tdcsfog_test["AccML_z"].abs() > threshold) | \
           (tdcsfog_test["AccAP_z"].abs() > threshold)

# Filter out these rows from the dataset
tdcsfog_test = tdcsfog_test[~outliers].drop(columns=["AccV_z", "AccML_z", "AccAP_z"])

# Save or inspect the filtered DataFrame
print("Number of points dropped:", outliers.sum())
print(tdcsfog_test.head())

Number of points dropped: 235
   Time      AccV     AccML     AccAP  StartHesitation  Turn  Walking
0     0 -9.851042 -0.111723 -1.172427                0     0        0
1     1 -9.849299 -0.116912 -1.173887                0     0        0
2     2 -9.847808 -0.115431 -1.176858                0     0        0
3     3 -9.848214 -0.103538 -1.177636                0     0        0
4     4 -9.850402 -0.104952 -1.179113                0     0        0


In [15]:
#filter the accelerometer data - defog training & testing

from scipy.signal import butter, filtfilt

fs = 100  # Sampling frequency in Hz

# Define filter parameters
lowcut = 0.25
highcut = 10.0
order = 4  # Filter order

# Create a Butterworth bandpass filter
nyquist = 0.5 * fs
low = lowcut / nyquist
high = highcut / nyquist
b, a = butter(order, [low, high], btype="band")

# Apply the filter to each accelerometer axis
defog["AccV_filtered"] = filtfilt(b, a, defog["AccV"])
defog["AccML_filtered"] = filtfilt(b, a, defog["AccML"])
defog["AccAP_filtered"] = filtfilt(b, a, defog["AccAP"])

defog_test["AccV_filtered"] = filtfilt(b, a, defog_test["AccV"])
defog_test["AccML_filtered"] = filtfilt(b, a, defog_test["AccML"])
defog_test["AccAP_filtered"] = filtfilt(b, a, defog_test["AccAP"])

print(defog.head())
print(defog_test.head())

      Time      AccV     AccML     AccAP  StartHesitation  Turn  Walking  \
1000  1000 -0.970018  0.061626 -0.265625                0     0        0   
1001  1001 -0.984375  0.044497 -0.265625                0     0        0   
1002  1002 -0.984375  0.029016 -0.265625                0     0        0   
1003  1003 -0.984375  0.015625 -0.265625                0     0        0   
1004  1004 -0.984670  0.015330 -0.265625                0     0        0   

      Valid  Task source_directory        csv_name  AccV_filtered  \
1000   True  True            defog  02ea782681.csv      -0.002409   
1001   True  True            defog  02ea782681.csv      -0.008514   
1002   True  True            defog  02ea782681.csv      -0.014268   
1003   True  True            defog  02ea782681.csv      -0.019375   
1004   True  True            defog  02ea782681.csv      -0.023621   

      AccML_filtered  AccAP_filtered  
1000       -0.028097        0.003932  
1001       -0.043759        0.003981  
1002       

In [16]:
#filter the accelerometer data - tdcsfog training and testing

from scipy.signal import butter, filtfilt

fs = 128  # Sampling frequency in Hz

# Define filter parameters
lowcut = 0.25
highcut = 10.0
order = 4  # Filter order

# Create a Butterworth bandpass filter
nyquist = 0.5 * fs
low = lowcut / nyquist
high = highcut / nyquist
b, a = butter(order, [low, high], btype="band")

# Apply the filter to each accelerometer axis
tdcsfog["AccV_filtered"] = filtfilt(b, a, tdcsfog["AccV"])
tdcsfog["AccML_filtered"] = filtfilt(b, a, tdcsfog["AccML"])
tdcsfog["AccAP_filtered"] = filtfilt(b, a, tdcsfog["AccAP"])

tdcsfog_test["AccV_filtered"] = filtfilt(b, a, tdcsfog_test["AccV"])
tdcsfog_test["AccML_filtered"] = filtfilt(b, a, tdcsfog_test["AccML"])
tdcsfog_test["AccAP_filtered"] = filtfilt(b, a, tdcsfog_test["AccAP"])

print(tdcsfog.head())

   Time      AccV     AccML     AccAP  StartHesitation  Turn  Walking  \
0     0 -9.533939  0.566322 -1.413525                0     0        0   
1     1 -9.536140  0.564137 -1.440621                0     0        0   
2     2 -9.529345  0.561765 -1.429332                0     0        0   
3     3 -9.531239  0.564227 -1.415490                0     0        0   
4     4 -9.540825  0.561854 -1.429471                0     0        0   

  source_directory        csv_name  AccV_filtered  AccML_filtered  \
0          tdcsfog  003f117e14.csv       0.001085        0.000976   
1          tdcsfog  003f117e14.csv       0.000987       -0.000830   
2          tdcsfog  003f117e14.csv       0.000909       -0.002732   
3          tdcsfog  003f117e14.csv       0.000867       -0.004787   
4          tdcsfog  003f117e14.csv       0.000862       -0.006999   

   AccAP_filtered  
0        0.006895  
1        0.003351  
2        0.000218  
3       -0.002184  
4       -0.003691  


In [17]:
defog = defog.drop(columns=['AccV', 'AccML', 'AccAP'])
defog_test = defog_test.drop(columns=['AccV', 'AccML', 'AccAP'])

tdcsfog = tdcsfog.drop(columns=['AccV', 'AccML', 'AccAP'])
tdcsfog_test = tdcsfog_test.drop(columns=['AccV', 'AccML', 'AccAP'])

In [18]:
defog = defog.reset_index(drop=True)
defog_test = defog_test.reset_index(drop=True)

tdcsfog = tdcsfog.reset_index(drop=True)
tdcsfog_test = tdcsfog_test.reset_index(drop=True)

In [19]:
defog = defog.drop(columns = ['Task', 'source_directory', 'Valid', 'csv_name', 'Time'])
defog_test = defog_test.drop(columns = ['Task', 'Valid', 'Time'])

In [20]:
tdcsfog = tdcsfog.drop(columns = ['source_directory', 'csv_name', 'Time'])
tdcsfog_test = tdcsfog_test.drop(columns = ['Time'])

In [21]:
print(defog.head())
print(defog_test.head())
print(tdcsfog.head())
print(tdcsfog_test.head())

   StartHesitation  Turn  Walking  AccV_filtered  AccML_filtered  \
0                0     0        0      -0.002409       -0.028097   
1                0     0        0      -0.008514       -0.043759   
2                0     0        0      -0.014268       -0.057533   
3                0     0        0      -0.019375       -0.068014   
4                0     0        0      -0.023621       -0.074621   

   AccAP_filtered  
0        0.003932  
1        0.003981  
2        0.004027  
3        0.004066  
4        0.004092  
   StartHesitation  Turn  Walking  AccV_filtered  AccML_filtered  \
0                0     0        0       0.000002       -0.000828   
1                0     0        0       0.003406       -0.000863   
2                0     0        0       0.005426       -0.001328   
3                0     0        0       0.005318       -0.002424   
4                0     0        0       0.003289       -0.004007   

   AccAP_filtered  
0       -0.000223  
1        0.000509  
2 

# Segment Data

In [22]:
sample_rate_tdcsfog = 128 # 128 samples per second
sample_rate_defog = 100

In [23]:
import numpy as np
import pandas as pd
from scipy.stats import mode

def segment_input_data(df, input_columns, sampling_rate, window_size_seconds=3, step_size_seconds=1):
    """
    Segments time-series input data (accelerometer values) into overlapping windows.

    Args:
        df (pd.DataFrame): Input DataFrame with time-series data.
        input_columns (list of str): List of column names for input data (e.g., accelerometer).
        window_size_seconds (int): Size of each window in seconds.
        step_size_seconds (int): Step size for sliding windows in seconds.
        sampling_rate (int): Sampling rate of the time-series data (e.g., 128 Hz).

    Returns:
        np.ndarray: Segmented input data of shape (num_windows, time_steps, input_features).
    """
    # Calculate the number of samples per window and step
    window_size_samples = int(window_size_seconds * sampling_rate)
    step_size_samples = int(step_size_seconds * sampling_rate)

    inputs = []

    # Segment the input data (accelerometer values)
    for start_idx in range(0, len(df) - window_size_samples + 1, step_size_samples):
        # Segment input columns (accelerometer values)
        input_window = df[input_columns].iloc[start_idx : start_idx + window_size_samples].values
        inputs.append(input_window)

    return np.array(inputs)

In [24]:
def segment_output_data(df, output_columns, sampling_rate, window_size_seconds=3, step_size_seconds=1):
    """
    Segments time-series output data (binary labels) into overlapping windows and aggregates them.

    Args:
        df (pd.DataFrame): Input DataFrame with time-series data.
        output_columns (list of str): List of column names for output data (e.g., binary labels).
        window_size_seconds (int): Size of each window in seconds.
        step_size_seconds (int): Step size for sliding windows in seconds.
        sampling_rate (int): Sampling rate of the time-series data (e.g., 128 Hz).

    Returns:
        np.ndarray: Segmented output data of shape (num_windows, output_features).
    """
    # Calculate the number of samples per window and step
    window_size_samples = int(window_size_seconds * sampling_rate)
    step_size_samples = int(step_size_seconds * sampling_rate)

    outputs = []

    # Segment the output data (binary labels)
    for start_idx in range(0, len(df) - window_size_samples + 1, step_size_samples):
        # Aggregate output columns (e.g., most common label within the window)
        output_window = df[output_columns].iloc[start_idx : start_idx + window_size_samples]
        output_values = mode(output_window, axis=0).mode[0]  # Most common value
        outputs.append(output_values)
    return np.array(outputs)

In [25]:
X = ['AccV_filtered', 'AccML_filtered', 'AccAP_filtered']
Turn = ["Turn"]
SH = ["StartHesitation"]
W = ["Walking"]

In [ ]:
defog_segments_X_train= segment_input_data(defog, X, sample_rate_defog)

In [ ]:
defog_segments_X_test= segment_input_data(defog_test, X, sample_rate_defog)

In [ ]:
defog_segments_y_train_T= segment_output_data(defog, Turn, sample_rate_defog)
defog_segments_y_train_SH= segment_output_data(defog, SH, sample_rate_defog)
defog_segments_y_train_W= segment_output_data(defog, W, sample_rate_defog)

In [1]:
defog_segments_y_test_T= segment_output_data(defog_test, Turn, sample_rate_defog)
defog_segments_y_test_SH= segment_output_data(defog_test, SH, sample_rate_defog)
defog_segments_y_test_W= segment_output_data(defog_test, W, sample_rate_defog)

NameError: name 'segment_output_data' is not defined

In [ ]:
print(defog_segments_X_train.shape)
print(defog_segments_X_test.shape)
print(defog_segments_y_train_T.shape)
print(defog_segments_y_test_T.shape)

(1974, 3000, 3)
(63, 3000, 3)
(1974,)
(63,)


In [29]:
tdcsfog_segments_X_train= segment_input_data(tdcsfog, X, sample_rate_tdcsfog)


KeyboardInterrupt: 

In [26]:
tdcsfog_segments_X_test= segment_input_data(tdcsfog_test, X, sample_rate_tdcsfog)

In [28]:
tdcsfog_segments_y_train_T= segment_output_data(tdcsfog, Turn, sample_rate_tdcsfog)
tdcsfog_segments_y_train_SH= segment_output_data(tdcsfog, SH, sample_rate_tdcsfog)
tdcsfog_segments_y_train_W= segment_output_data(tdcsfog, W, sample_rate_tdcsfog)

In [27]:
tdcsfog_segments_y_test_T= segment_output_data(tdcsfog_test, Turn, sample_rate_tdcsfog)
tdcsfog_segments_y_test_SH= segment_output_data(tdcsfog_test, SH, sample_rate_tdcsfog)
tdcsfog_segments_y_test_W= segment_output_data(tdcsfog_test, W, sample_rate_tdcsfog)

# Model

In [ ]:
import os
os.environ["TORCH_INDUCTOR_DISABLE"] = "1"
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#https://github.com/YasinShafiei/ResNet-From-Scratch/blob/main/architectures.py
#https://github.com/janbrederecke/fog/blob/main/src/model.py

In [ ]:
def _padding(downsample, kernel_size):
    """Compute required padding"""
    padding = (kernel_size - 1) // 2
    return padding

In [ ]:
def _downsample(n_samples_in, n_samples_out):
    """Compute downsample rate"""
    downsample = int(n_samples_in // n_samples_out)
    if downsample < 1:
        raise ValueError("Number of samples should always decrease")
    if n_samples_in % n_samples_out != 0:
        raise ValueError(
            "Number of samples for two consecutive blocks "
            "should always decrease by an integer factor."
        )
    return downsample


In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample, kernel_size, dropout_rate):
        if kernel_size % 2 == 0:
            raise ValueError("This implementation only supports odd `kernel_size`.")
        super(BasicBlock, self).__init__()

        # Forward path
        padding = _padding(1, kernel_size)

        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.LeakyReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        padding = _padding(downsample, kernel_size)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=padding, stride=downsample, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.dropout2 = nn.Dropout(dropout_rate)

        # Skip connection
        skip_connection_layers = []
        if downsample > 1:
            maxpool = nn.MaxPool1d(downsample, stride=downsample)
            skip_connection_layers.append(maxpool)

        if in_channels != out_channels:
            conv1x1 = nn.Conv1d(in_channels, out_channels, 1, bias=False)
            skip_connection_layers.append(conv1x1)

        self.skip_connection = nn.Sequential(*skip_connection_layers) if skip_connection_layers else None

    def forward(self, x):
        # Apply skip connection (if needed)
        if self.skip_connection is not None:
            y = self.skip_connection(x)
        else:
            y = x  # No change to input if no skip connection

        # 1st layer
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout1(x)

        # 2nd layer
        x = self.conv2(x)

        # Add skip connection (if present)
        x += y

        # Final batch norm, ReLU, and dropout
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout2(x)

        return x


In [ ]:
#Residual Network for 1D accelerometer data.
class SmallResNet(nn.Module):
    def __init__(self, input_dim, blocks_dim, n_classes, kernel_size=17, dropout_rate=0.8):
        super(SmallResNet, self).__init__()

        # First layers
        in_channels, out_channels = input_dim[0], blocks_dim[0][0]
        downsample = 1
        padding = (kernel_size - 1) // 2  # Adjust padding for odd kernel size

        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, bias=False, stride=downsample, padding=padding)
        self.bn1 = nn.BatchNorm1d(out_channels)

        # Residual block layers
        self.res_blocks = nn.ModuleList()
        for i, (n_filters, n_samples) in enumerate(blocks_dim):
            in_channels, out_channels = out_channels, n_filters
            resblk1d = BasicBlock(in_channels, n_filters, downsample, kernel_size, dropout_rate)
            self.res_blocks.append(resblk1d)

        # Linear layer
        n_filters_last, n_samples_last = blocks_dim[-1]
        self.lin = nn.Linear(n_filters_last, n_classes)
        self.n_blk = len(blocks_dim)

    def forward(self, x):
        # First layers
        x = self.conv1(x)
        x = self.bn1(x)

        # Residual blocks
        y = x
        for blk in self.res_blocks:
            x = blk(x)  # Only pass the input tensor, not the skip connection

        # Flatten array
        x = x.mean(-1)

        logits = self.lin(x)
        return logits


# Deploy Model

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [ ]:
class AccelerometerBinaryDataset(Dataset):
    def __init__(self, data, labels):
        """
        Args:
            data (numpy array or tensor): Shape (num_samples, sequence_length)
            labels (numpy array or tensor): Shape (num_samples,)
        """
        # Convert DataFrame to NumPy array if necessary
        if isinstance(data, pd.DataFrame):
            data = data.values  # Convert DataFrame to NumPy array
        if isinstance(labels, (pd.DataFrame, pd.Series)):
            labels = labels.values

        # Ensure data is 2D (num_samples, sequence_length)
        if data.ndim == 2:  # Use ndim for numpy arrays
            data = np.expand_dims(data, axis=1)  # Add a channel dimension: shape becomes (num_samples, 1, sequence_length)

        # Convert to PyTorch tensors
        self.data = torch.tensor(data, dtype=torch.float32)  # Shape (num_samples, 1, sequence_length)
        self.labels = torch.tensor(labels, dtype=torch.long)  # Shape (num_samples,)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


In [ ]:
# Model initialization
input_dim = (3, sample_rate_defog)  # 3 input features
blocks_dim = [(16, 1), (32, 1), (64, 1)]  # Example block dimensions
n_classes = 3
model = SmallResNet(input_dim=input_dim, blocks_dim=blocks_dim, n_classes=n_classes,  kernel_size=17,dropout_rate=0.0,)


In [ ]:
# Check input and output shapes
test_input = torch.randn(32, 3, 128)  # Batch of 32, 3 channels, sequence length 128
test_output = model(test_input)
print(f"Output shape: {test_output.shape}")



Output shape: torch.Size([32, 3])


In [ ]:
criterion = nn.CrossEntropyLoss()  # For multi-class or binary classification

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
print(model)

SmallResNet(
  (conv1): Conv1d(3, 16, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
  (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (res_blocks): ModuleList(
    (0): BasicBlock(
      (conv1): Conv1d(16, 16, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
      (bn1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): LeakyReLU(negative_slope=0.01)
      (dropout1): Dropout(p=0.0, inplace=False)
      (conv2): Conv1d(16, 16, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
      (bn2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout2): Dropout(p=0.0, inplace=False)
    )
    (1): BasicBlock(
      (conv1): Conv1d(16, 32, kernel_size=(17,), stride=(1,), padding=(8,), bias=False)
      (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): LeakyReLU(negative_slope=0.01)
      (dro

In [ ]:
def train_model(model, train_loader, criterion, optimizer, epochs=10):
# Training loop
  num_epochs = 3  # Number of epochs
  for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs_batch, labels_batch in train_loader:
        inputs_batch = inputs_batch.permute(0, 2, 1)  # Change shape from [batch_size, num_time_steps, num_channels] to [batch_size, num_channels, num_time_steps]

        optimizer.zero_grad()  # Zero gradients from previous iteration

        # Forward pass
        outputs = model(inputs_batch)

        # Compute loss
        loss = criterion(outputs, labels_batch)
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels_batch.size(0)
        correct += (predicted == labels_batch).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

In [ ]:
# Shape the data for defog turn
train_dataset_defog_Turn = AccelerometerBinaryDataset(defog_segments_X_train, defog_segments_y_train_T)
test_dataset_defog_Turn = AccelerometerBinaryDataset(defog_segments_X_test, defog_segments_y_test_T)

In [ ]:
train_loader_defog_Turn = DataLoader(train_dataset_defog_Turn, batch_size=32, shuffle=True)
test_loader_defog_Turn = DataLoader(test_dataset_defog_Turn, batch_size=32, shuffle=False)

In [ ]:
# Shape the data for defog walking
train_dataset_defog_W = AccelerometerBinaryDataset(defog_segments_X_train, defog_segments_y_train_W)
test_dataset_defog_W = AccelerometerBinaryDataset(defog_segments_X_test, defog_segments_y_test_W)

train_loader_defog_W = DataLoader(train_dataset_defog_W, batch_size=32, shuffle=True)
test_loader_defog_W = DataLoader(test_dataset_defog_W, batch_size=32, shuffle=False)

In [ ]:
# Shape the data for defog StartHesitation
train_dataset_defog_SH = AccelerometerBinaryDataset(defog_segments_X_train, defog_segments_y_train_SH)
test_dataset_defog_SH = AccelerometerBinaryDataset(defog_segments_X_test, defog_segments_y_test_SH)

train_loader_defog_SH = DataLoader(train_dataset_defog_SH, batch_size=32, shuffle=True)
test_loader_defog_SH= DataLoader(test_dataset_defog_SH, batch_size=32, shuffle=False)

In [ ]:
#Shape data for tdcsfog Turn
train_dataset_tdcsfog_Turn = AccelerometerBinaryDataset(tdcsfog_segments_X_train, tdcsfog_segments_y_train_T)
train_loader_tdcsfog_Turn = DataLoader(train_dataset_tdcsfog_Turn, batch_size=32, shuffle=False)

test_dataset_tdcsfog_Turn = AccelerometerBinaryDataset(tdcsfog_segments_X_test, tdcsfog_segments_y_test_T)
test_loader_tdcsfog_Turn = DataLoader(test_dataset_tdcsfog_Turn, batch_size=32, shuffle=False)

In [ ]:
#Shape data for tdcsfog StartHesitation
train_dataset_tdcsfog_SH = AccelerometerBinaryDataset(tdcsfog_segments_X_train, tdcsfog_segments_y_train_SH)
train_loader_tdcsfog_SH = DataLoader(train_dataset_tdcsfog_SH, batch_size=32, shuffle=False)

test_dataset_tdcsfog_SH = AccelerometerBinaryDataset(tdcsfog_segments_X_test, tdcsfog_segments_y_test_SH)
test_loader_tdcsfog_SH = DataLoader(test_dataset_tdcsfog_SH, batch_size=32, shuffle=False)

In [ ]:
#Shape data for tdcsfog Walking
train_dataset_tdcsfog_W = AccelerometerBinaryDataset(tdcsfog_segments_X_train, tdcsfog_segments_y_train_W)
train_loader_tdcsfog_W= DataLoader(train_dataset_tdcsfog_W, batch_size=32, shuffle=False)

test_dataset_tdcsfog_W = AccelerometerBinaryDataset(tdcsfog_segments_X_test, tdcsfog_segments_y_test_W)
test_loader_tdcsfog_W= DataLoader(test_dataset_tdcsfog_W, batch_size=32, shuffle=False)

In [ ]:
#Train model
train_defog_turn_model = train_model(model, train_loader_defog_Turn, criterion, optimizer, epochs= 3)

Epoch [1/3], Loss: 0.4998, Accuracy: 87.59%
Epoch [2/3], Loss: 0.4722, Accuracy: 87.59%
Epoch [3/3], Loss: 0.4513, Accuracy: 87.59%


In [ ]:
train_defog_W_model = train_model(model, train_loader_defog_W, criterion, optimizer, epochs = 3)

Epoch [1/3], Loss: 0.9685, Accuracy: 78.32%
Epoch [2/3], Loss: 0.7007, Accuracy: 98.83%
Epoch [3/3], Loss: 0.5325, Accuracy: 98.83%


In [ ]:
train_defog_SH_model = train_model(model, train_loader_defog_SH, criterion, optimizer, epochs = 3)

Epoch [1/3], Loss: 0.8859, Accuracy: 88.45%
Epoch [2/3], Loss: 0.6481, Accuracy: 100.00%
Epoch [3/3], Loss: 0.4885, Accuracy: 100.00%


In [ ]:
train_tdcsfog_turn_model = train_model(model, train_loader_tdcsfog_Turn, criterion, optimizer, epochs = 3)

In [ ]:
train_tdcsfog_W_model = train_model(model, train_loader_tdcsfog_W, criterion, optimizer, epochs = 3)

In [ ]:
train_tdcsfog_SH_model = train_model(model, train_loader_tdcsfog_SH, criterion, optimizer, epochs = 3)

# Model Evaluation

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_curve


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    all_preds = []  # Store all predictions
    all_labels = []  # Store all true labels

    with torch.no_grad():  # No need to track gradients during evaluation
        for inputs_batch, labels_batch in test_loader:
            inputs_batch = inputs_batch.permute(0, 2, 1)  # Ensure correct shape for Conv1d

            # Forward pass
            outputs = model(inputs_batch)

            # If binary classification, apply sigmoid and threshold
            if outputs.shape[1] == 1:  # Single output for binary classification
                predicted = (outputs > 0.5).cpu().numpy()  # Apply threshold
            else:
                _, predicted = torch.max(outputs, 1)  # Multi-class classification

            # Collect predictions and true labels
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels_batch.cpu().numpy())

    # Calculate Precision, Recall, and Weighted F1 Score
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)

    # Print the metrics
    print(f"Precision (Weighted): {precision:.4f}")
    print(f"Recall (Weighted): {recall:.4f}")
    print(f"F1 Score (Weighted): {f1:.4f}")


In [ ]:
# Assuming `model` is your trained model and `test_loader` is your DataLoader for the test dataset
evaluate_model(model, test_loader_defog_Turn)

Precision (Weighted): 0.7727
Recall (Weighted): 0.6508
F1 Score (Weighted): 0.5131


In [ ]:
#Defog Walking
print("Evaluation for Walking defog events in Defog")
evaluate_model(model, test_loader_defog_W)

Evaluation for Walking defog events in Defog
Precision (Weighted): 1.0000
Recall (Weighted): 1.0000
F1 Score (Weighted): 1.0000


In [ ]:
#Defog Walking
print("Evaluation for SH defog events in Defog")
evaluate_model(model, test_loader_defog_SH)

Evaluation for SH defog events in Defog
Precision (Weighted): 1.0000
Recall (Weighted): 1.0000
F1 Score (Weighted): 1.0000


In [ ]:
#Tdcsfog Turn
print("Evaluation for Turn FOG events in tdcsfog")
evaluate_model(model, test_loader_tdcsfog_Turn)

Evaluation for Turn FOG events in tdcsfog
Precision (Weighted): 0.7578
Recall (Weighted): 0.5882
F1 Score (Weighted): 0.4357


In [ ]:
#Tdcsfog SH
print("Evaluation for SH FOG events in tdcsfog")
evaluate_model(model, test_loader_tdcsfog_SH)

Evaluation for SH FOG events in tdcsfog
Precision (Weighted): 0.7924
Recall (Weighted): 0.2941
F1 Score (Weighted): 0.1337


In [ ]:
#Tdcsfog SH
print("Evaluation for W FOG events in tdcsfog")
evaluate_model(model, test_loader_tdcsfog_W)

Evaluation for W FOG events in tdcsfog
Precision (Weighted): 0.8962
Recall (Weighted): 0.1176
F1 Score (Weighted): 0.0248
